# 10 — Enforcement Action Effectiveness (Causal Analysis)

This notebook estimates the **causal effect** of each enforcement action type on
network latency using three methods:

1. **Pre / Post Window Analysis** — mean & p95 latency changes, restoration time
2. **Difference-in-Differences (DiD)** — treatment vs. control comparison
3. **Propensity-Score Matching (PSM)** — matched causal ATE estimates

Outputs: `reports/enforcement_analysis.md`, `reports/enforcement_effects_summary.csv`,
and three figures.

In [ ]:
import sys, pathlib
ROOT = pathlib.Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src.models.enforcement_effects import (
    load_data, pre_post_analysis, did_analysis, psm_analysis,
    plot_pre_post, plot_did_psm, plot_effectiveness_heatmap,
    save_results, WINDOW_NS,
)
import pandas as pd, numpy as np
from IPython.display import display, Markdown, Image

print("Imports OK")

## 1. Load data

In [ ]:
df = load_data()
enforced = df[df["action_type"].notna()]
print(f"Total rows: {len(df):,}")
print(f"Enforcement events: {len(enforced)}")
print(f"\nAction types:\n{enforced['action_type'].value_counts().to_string()}")
print(f"\nAttack types in enforced:\n{enforced['attack_type'].value_counts().to_string()}")

## 2. Pre / Post Window Analysis

In [ ]:
pp_df = pre_post_analysis(df)
print(f"Events with sufficient window data: {len(pp_df)}")
print(f"Window: ±{WINDOW_NS/1000:.0f} μs\n")

display(pp_df.groupby("action_type")[["delta_mean", "delta_p95", "restoration_time_ns"]]
        .agg(["mean", "std"]).round(4))

In [ ]:
plot_pre_post(pp_df)
display(Image(str(ROOT / "figures" / "enforcement_pre_post.png")))

## 3. Difference-in-Differences

In [ ]:
did_df = did_analysis(df)
display(did_df.round(4))

## 4. Propensity-Score Matching

In [ ]:
psm_df, treated, matched_ctrl = psm_analysis(df)
display(psm_df.round(4))

# Propensity score overlap
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 3))
ax.hist(treated["propensity"], bins=40, alpha=0.6, label="Treated", color="coral")
ax.hist(matched_ctrl["propensity"], bins=40, alpha=0.6, label="Matched Control", color="steelblue")
ax.set_xlabel("Propensity Score"); ax.set_ylabel("Count")
ax.set_title("Propensity-Score Overlap"); ax.legend()
plt.tight_layout(); plt.show()

## 5. ATE Forest Plot (DiD + PSM)

In [ ]:
plot_did_psm(did_df, psm_df)
display(Image(str(ROOT / "figures" / "enforcement_ate_forest.png")))

## 6. Effectiveness Heatmap  (Action × Attack Type)

In [ ]:
plot_effectiveness_heatmap(pp_df)
display(Image(str(ROOT / "figures" / "enforcement_effectiveness_heatmap.png")))

## 7. Save reports & summary CSV

In [ ]:
csv_path, md_path = save_results(pp_df, did_df, psm_df)
print(f"\nCSV : {csv_path}")
print(f"MD  : {md_path}")

## 8. Display full report

In [ ]:
display(Markdown((ROOT / "reports" / "enforcement_analysis.md").read_text(encoding="utf-8")))